In [ ]:
import polars as pl

texas_counties = [
    'Anderson', 'Andrews', 'Angelina', 'Aransas', 'Archer', 'Armstrong', 'Atascosa', 'Bailey', 'Bandera', 
    'Bastrop', 'Baylor', 'Bee', 'Bell', 'Bexar', 'Blanco', 'Borden', 'Bosque', 'Bowie', 'Brazoria', 'Brazos', 
    'Brewster', 'Briscoe', 'Brooks', 'Brown', 'Burleson', 'Burnet', 'Caldwell', 'Calhoun', 'Callahan', 'Cameron', 
    'Camp', 'Carson', 'Cass', 'Castro', 'Chambers', 'Cherokee', 'Childress', 'Clay', 'Cochran', 'Coke', 'Coleman', 
    'Collin', 'Collingsworth', 'Colorado', 'Comal', 'Comanche', 'Concho', 'Cooke', 'Coryell', 'Cottle', 'Crane', 
    'Crockett', 'Crosby', 'Culberson', 'Dallam', 'Dallas', 'Dawson', 'Deaf Smith', 'Delta', 'Denton', 'DeWitt', 
    'Dickens', 'Dimmit', 'Donley', 'Duval', 'Eastland', 'Ector', 'Edwards', 'Ellis', 'Erath', 'Falls', 
    'Fannin', 'Fayette', 'Fisher', 'Floyd', 'Foard', 'Fort Bend', 'Franklin', 'Freestone', 'Frio', 'Gaines', 
    'Galveston', 'Garza', 'Gillespie', 'Glasscock', 'Goliad', 'Gonzales', 'Gray', 'Grayson', 'Gregg', 'Grimes', 
    'Guadalupe', 'Hale', 'Hall', 'Hamilton', 'Hansford', 'Hardeman', 'Hardin', 'Harris', 'Harrison', 'Hartley', 
    'Haskell', 'Hays', 'Hemphill', 'Henderson', 'Hidalgo', 'Hill', 'Hockley', 'Hood', 'Hopkins', 'Howard', 
    'Hudspeth', 'Hunt', 'Hutchinson', 'Irion', 'Jack', 'Jackson', 'Jasper', 'Jeff Davis', 'Jefferson', 'Jim Hogg', 
    'Jim Wells', 'Johnson', 'Jones', 'Karnes', 'Kaufman', 'Kendall', 'Kenedy', 'Kent', 'Kerr', 'Kimble', 'King', 
    'Kinney', 'Kleberg', 'Knox', 'Lamar', 'Lamb', 'Lampasas', 'La Salle', 'Lavaca', 'Lee', 'Leon', 'Liberty', 
    'Limestone', 'Lipscomb', 'Live Oak', 'Llano', 'Loving', 'Lynn', 'McCulloch', 'McLennan', 'McMullen', 
    'Madison', 'Marion', 'Martin', 'Mason', 'Matagorda', 'Maverick', 'Medina', 'Menard', 'Midland', 'Milam', 
    'Mills', 'Mitchell', 'Montague', 'Montgomery', 'Moore', 'Morris', 'Motley', 'Nacogdoches', 'Navarro', 
    'Newton', 'Nolan', 'Nueces', 'Ochiltree', 'Oldham', 'Orange', 'Palo Pinto', 'Panola', 'Parker', 'Parmer', 
    'Pecos', 'Polk', 'Potter', 'Presidio', 'Rains', 'Randall', 'Reagan', 'Real', 'Red River', 'Reeves', 'Refugio', 
    'Roberts', 'Robertson', 'Rockwall', 'Runnels', 'Rusk', 'Sabine', 'San Augustine', 'San Jacinto', 'San Patricio', 
    'San Saba', 'Schleicher', 'Scurry', 'Shackelford', 'Shelby', 'Sherman', 'Smith', 'Somervell', 'Starr', 
    'Stephens', 'Sterling', 'Stonewall', 'Sutton', 'Swisher', 'Tarrant', 'Taylor', 'Terrell', 'Terry', 'Throckmorton', 
    'Titus', 'Tom Green', 'Travis', 'Trinity', 'Tyler', 'Upshur', 'Upton', 'Uvalde', 'Val Verde', 'Van Zandt', 
    'Victoria', 'Walker', 'Waller', 'Ward', 'Washington', 'Webb', 'Wharton', 'Wheeler', 'Wichita', 'Wilbarger', 
    'Willacy', 'Williamson', 'Wilson', 'Winkler', 'Wise', 'Wood', 'Yoakum', 'Young', 'Zapata', 'Zavala'
]

texas_counties_df = pl.DataFrame({"county": texas_counties})
texas_counties_df.write_csv('texas_counties.csv')
print('success!')

In [ ]:
texas_counties_df = texas_counties_df.with_columns( pl.col("county").str.to_uppercase())
texas_counties_df

In [ ]:
from src.pool import PG

ROUTES_QUERY = """--sql 
    SELECT route_name FROM properties_dba.routes
"""

routes_df = await PG.fetch("SELECT route_id, route_name, division_id FROM properties_dba.routes")
if routes_df is None:
    raise ValueError("df is None")
if routes_df.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

routes_df

In [ ]:
from src.pool import PG

FACILITIES_QUERY = """--sql 
    SELECT *
    FROM properties_dba.mrte_facility
    WHERE primo_prprty NOT IN (69419, 98750, 98743);
"""

facilities_df = await PG.fetch(FACILITIES_QUERY)
if facilities_df is None:
    raise ValueError("df is None")
if facilities_df.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

facilities_df

In [ ]:
def map_route_to_division(route_df: pl.DataFrame, division_id: int) -> str:
    get_divsion = route_df.filter(pl.col("division_id") == division_id)
    route_assignment = random.choice(get_divsion)
    route_chosen = route_assignment.select("route_id").item()
    return route_chosen

In [ ]:
map_route_to_division(routes_df, 12)

In [ ]:
pl.Config.set_tbl_rows(1500)

In [ ]:
#facility gen

import random
from datetime import datetime, timedelta

def map_route_to_division(route_df: pl.DataFrame, division_id: int):
    get_divsion = route_df.filter(pl.col("division_id") == division_id)
    route_assignment = random.choice(get_divsion)
    route_chosen = route_assignment.select("route_id").item()
    return route_chosen

def testing_data() -> pl.DataFrame:
    county_row_count = texas_counties_df.shape[0]
    
    facilities = []

    facility_id = 348823750
    primo_prprty = 98760

    batch_size = 10
    start_index = 0

    for route in routes_df.iter_rows(named=True):
        division_id = route['division_id']

        # Generate 10 facilities for each route
        for i in range(1, 11):
            # Ensure start_index wraps around if it exceeds county_count
            county_index = (start_index + i - 1) % county_row_count
            county_name = texas_counties_df["county"][county_index]

            facility_id += 1
            primo_prprty +=1
            route_id = map_route_to_division(routes_df, division_id)

            end_date = datetime.now()
            start_date = end_date - timedelta(days=365 * 10)
            first_production_ts = start_date + (end_date - start_date) * random.random()

            facilities.append({
                'facility_id': facility_id,
                'facility_name': f'{county_name} FAC',
                'first_production_ts': first_production_ts,    
                'division_id': division_id,
                'primo_prprty': primo_prprty,
                'route_id': route_id
            })

        start_index = (start_index + batch_size) % county_row_count


    facilities_df = pl.DataFrame(facilities)
    return facilities_df

In [ ]:
testing_data()

In [ ]:
import polars as pl

facilities_df = facilities_df.with_columns(pl.col("primo_prprty").cast(str))
facilities_df

In [ ]:
#tank gen
import random

import uuid


def tank_generation() -> pl.DataFrame:
    tanks = []

    for facs in facilities_df.iter_rows(named=True):
        division_id = facs['division_id']
        primo_prprty =  facs['primo_prprty']

        range_oil_tanks = random.randint(2,6)
        range_water_tanks = random.randint(1,3)


        #generating oil tanks
        for i in range(1, range_oil_tanks + 1):
            tanks.append({
                'key_metric': uuid.uuid4(),
                'source_key': primo_prprty + '0' + str(i),
                'source_id': primo_prprty,
                'metric_nice_name': 'OilTank' + str(i) + 'Level',
                'uom': 'in',
                'division_id': division_id
            })
            tanks.append({
                'key_metric': uuid.uuid4(),
                'source_key': primo_prprty + '0' + str(i),
                'source_id': primo_prprty,
                'metric_nice_name': 'OilTank' + str(i) + 'Volume',
                'uom': 'bbl',
                'division_id': division_id
            })

        # generating water tanks
        for i in range(1, range_water_tanks + 1):
            tanks.append({
                'key_metric': uuid.uuid4(),
                'source_key': primo_prprty + 'W' + str(i),
                'source_id': primo_prprty,
                'metric_nice_name': 'WaterTank' + str(i) + 'Level',
                'uom': 'in',
                'division_id': division_id
            })
            tanks.append({
                'key_metric': uuid.uuid4(),
                'source_key': primo_prprty + 'W' + str(i),
                'source_id': primo_prprty,
                'metric_nice_name': 'WaterTank' + str(i) + 'Volume',
                'uom': 'bbl',
                'division_id': division_id
            })

        tanks.append({
                'key_metric': uuid.uuid4(),
                'source_key': primo_prprty + 'FAC',
                'source_id': primo_prprty,
                'metric_nice_name': 'ESD-OilTankID',
                'uom': '',
                'division_id': division_id
        })

        tanks.append({
            'key_metric': uuid.uuid4(),
            'source_key': primo_prprty + 'FAC',
            'source_id': primo_prprty,
            'metric_nice_name': 'ESD-OilTankInchesUntilAlarm',
            'uom': 'in',
            'division_id': division_id
        })

        tanks.append({
            'key_metric': uuid.uuid4(),
            'source_key': primo_prprty + 'FAC',
            'source_id': primo_prprty,
            'metric_nice_name': 'ESD-WaterTankID',
            'uom': '',
            'division_id': division_id
        })

        tanks.append({
            'key_metric': uuid.uuid4(),
            'source_key': primo_prprty + 'FAC',
            'source_id': primo_prprty,
            'metric_nice_name': 'ESD-WaterTankInchesUntilAlarm',
            'uom': 'in',
            'division_id': division_id
        })

    tanks_df = pl.DataFrame(tanks)
    return tanks_df

In [ ]:
pl.Config.set_tbl_rows(1500)

In [ ]:
tank_generation()

In [ ]:
from src.pool import PG

await PG.insert(tank_generation(), "timeseries_dba.data_catalog")

In [ ]:
from src.pool import PG

NEW_TANKS_QUERY = """--sql 
    SELECT DISTINCT source_key, division_id
    FROM timeseries_dba.data_catalog dc
    WHERE source_id NOT IN ('69419', '98750', '98743', '480001', '110009')
    AND source_key NOT LIKE '%FAC%'
    ORDER BY source_key;
"""

new_tanks_df = await PG.fetch(NEW_TANKS_QUERY)
if new_tanks_df is None:
    raise ValueError("df is None")
if new_tanks_df.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

new_tanks_df

In [ ]:
#tank_metadata
import polars as pl
from datetime import datetime, timedelta
import random
from decimal import Decimal

def tank_timeseries_generation() -> pl.DataFrame:
    tanks_ts = []

    for i in new_tanks_df.iter_rows(named=True):
        source_key = i["source_key"]
        division_id = i["division_id"]
        
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365 * 5)
        create_ts = start_date + (end_date - start_date) * random.random()

        tanksize = random.uniform(400, 800)

        tanks_ts.append({
            "scadaid": source_key,
            "create_ts": create_ts, 
            "update_ts": create_ts,
            "tanksize": tanksize,
            "division_id": division_id
        })

    tanks_df = pl.DataFrame(tanks_ts)
    return tanks_df


In [ ]:
tank_timeseries_generation()

In [ ]:
from src.pool import PG

await PG.insert(tank_timeseries_generation(), "tank.tank_metadata")

In [ ]:
from src.pool import PG

DATA_CATALOG_QUERY = """--sql 
    SELECT *
    FROM timeseries_dba.data_catalog dc
    WHERE source_id NOT IN ('69419', '98750', '98743', '480001', '110009')
"""

dc_df = await PG.fetch(DATA_CATALOG_QUERY)
if dc_df is None:
    raise ValueError("df is None")
if dc_df.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

dc_df

In [ ]:
from src.pool import PG

NEW_TANKS_QUERY = """--sql 
    SELECT DISTINCT source_key, division_id, source_id
    FROM timeseries_dba.data_catalog dc
    WHERE source_id NOT IN ('69419', '98750', '98743', '480001', '110009')
    AND source_key NOT LIKE '%FAC%'
    ORDER BY source_key;
"""

new_tanks_df = await PG.fetch(NEW_TANKS_QUERY)
if new_tanks_df is None:
    raise ValueError("df is None")
if new_tanks_df.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

new_tanks_df

In [ ]:
pattern = r'0\d'
oil_tanks_df = new_tanks_df.filter(pl.col("source_key").str.contains(pattern)) 
new_oil_df = oil_tanks_df.group_by("source_id").agg(pl.count("source_key").alias("oil_tank_count"))
new_oil_df = new_oil_df.sort("source_id")
new_oil_df

In [ ]:
water_tanks_df = new_tanks_df.filter(pl.col("source_key").str.contains("W"))
new_df = water_tanks_df.group_by("source_id").agg(pl.count("source_key").alias("water_tank_count"))
new_df = new_df.sort("source_id")
new_df

In [ ]:
combined_count_df = new_df.join(new_oil_df, on="source_id")
combined_count_df

In [ ]:
import polars as pl
pl.Config.set_tbl_rows(2000)

In [ ]:
def select_rand_oil(new_df: pl.DataFrame, key_metric):
    oil_count = new_df.with_columns("oil_tank_count").item()
    return oil_count

In [ ]:
new_new_df = dc_df.join(combined_count_df, on="source_id")
new_new_df

In [ ]:
#timeseries
import polars as pl
import random
from datetime import datetime, timedelta

def tank_timeseries_with_value() -> pl.DataFrame:
    time_series = []
    value = 0 

    for tank in new_new_df.iter_rows(named=True):
        key_metric = tank["key_metric"]
        primo_id = tank["source_key"]
        identifier = tank["metric_nice_name"]

        if "Inches" in identifier:
            if "Oil" in identifier:
                value = random.randint(1, tank["oil_tank_count"])
            else:
                value = random.randint(1, tank["water_tank_count"])
        else: continue
        
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365)
        create_ts = start_date + (end_date - start_date) * random.random()

        time_series.append({
            "key_metric": key_metric,
            "pp": primo_id,
            "poop": identifier,
            "ts": create_ts,
            "value": value
        })

    time_series_df = pl.DataFrame(time_series)
    return time_series_df

In [ ]:
tank_timeseries_with_value()

In [ ]:
from src.pool import PG

NEW_TANKS_QUERY = """--sql 
    SELECT *
    FROM timeseries_dba.data_catalog dc
    WHERE source_id NOT IN ('69419', '98750', '98743', '480001', '110009')
    AND source_key NOT LIKE '%FAC%'
    ORDER BY source_key;
"""

wee = await PG.fetch(NEW_TANKS_QUERY)
if wee is None:
    raise ValueError("df is None")
if wee.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

wee

In [ ]:
#timeseries
import polars as pl
import random
from datetime import datetime, timedelta

def tank_timeseries_with_value2() -> pl.DataFrame:
    time_series = []

    for tank in wee.iter_rows(named=True):
        key_metric = tank["key_metric"]
        identifier = tank["metric_nice_name"]

        if "Volume" in identifier:
            value = random.uniform(120, 480)
        elif "Level" in identifier:
            value = random.uniform(60, 98)
        else:
            value = random.uniform(60, 400)

        end_date = datetime.now()
        start_date = end_date - timedelta(days=365 * 10)
        create_ts = start_date + (end_date - start_date) * random.random()

        time_series.append({
            "key_metric": key_metric,
            "ts": create_ts,
            "value": value
        })

    time_series_df = pl.DataFrame(time_series)
    return time_series_df

In [ ]:
tank_timeseries_with_value2()

In [ ]:
import polars as pl
import random
from datetime import datetime, timedelta
from perlin_noise import PerlinNoise

# Initialize Perlin noise generator
random_noise = PerlinNoise(octaves=1)

# Function to create the initial tank time series data
def tank_timeseries_with_value2() -> pl.DataFrame:
    time_series = []

    for tank in wee.iter_rows(named=True):
        key_metric = tank["key_metric"]
        identifier = tank["metric_nice_name"]

        end_date = datetime.now()
        start_date = end_date - timedelta(days=365 * 10)
        create_ts = start_date + (end_date - start_date) * random.random()

        # Generate value using Perlin noise
        if "Volume" in identifier:
            value = 300 + random_noise.noise(create_ts.timestamp() / 100000) * 180
        elif "Level" in identifier:
            value = 79 + random_noise.noise(create_ts.timestamp() / 100000) * 19
        else:
            value = 230 + random_noise.noise(create_ts.timestamp() / 100000) * 170

        time_series.append({
            "key_metric": key_metric,
            "ts": create_ts,
            "value": value
        })

    time_series_df = pl.DataFrame(time_series)
    return time_series_df

# Function to apply Perlin noise over time series
def create_ts_df_for_min(initial_df: pl.DataFrame, hour: int, total_minutes: int) -> pl.DataFrame:
    offset = timedelta(hours=hour)
    noise = random_noise.noise(hour / total_minutes)
    return initial_df.with_columns(
        pl.col("key_metric"),
        pl.col("ts") - offset,
        pl.col("value") + noise
    )

# Generate the time series data with Perlin noise
points_per_day = 24
days = 2
hours = points_per_day * days

# Use the fetched DataFrame for time series generation
initial_df = tank_timeseries_with_value2()
noise_dfs = map(
    lambda x: create_ts_df_for_min(initial_df, x, hours), range(hours)
)
noise_dfs = list(noise_dfs)

# Combine all generated data into one DataFrame
combined_df = pl.concat(noise_dfs)


In [ ]:
combined_df

In [ ]:
print(wee.columns)

In [ ]:
def create_ts_df_for_min(initial_df: pl.DataFrame, hour: int, total_minutes: int) -> pl.DataFrame:
    offset = timedelta(hours=hour)
    noise = random_noise.noise(hour / total_minutes)
    return initial_df.with_columns(
        pl.col("key_metric"),
        pl.col("ts") - offset,
        pl.col("value") + noise
    )


In [ ]:
from src.pool import PG 

DATA_CATALOG_QUERY = """--sql
    SELECT *
    FROM timeseries_dba.data_catalog AS dc 
    WHERE dc.metric_nice_name NOT LIKE '%TankID%'
    AND dc.metric_nice_name NOT LIKE '%TimeUntilESD%'
"""

dc = await PG.fetch(DATA_CATALOG_QUERY)
if dc is None:
    raise ValueError("df is None")
if dc.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

dc

In [ ]:
#facility gen

import random
from datetime import datetime, timedelta


def testing_data() -> pl.DataFrame:
    
    facilities = []

    for route in routes_df.iter_rows(named=True):
        division_id = route['division_id']

        # Generate 10 facilities for each route
        for i in range(1, 11):
            # Ensure start_index wraps around if it exceeds county_count
            county_index = (start_index + i - 1) % county_row_count
            county_name = texas_counties_df["county"][county_index]

            facility_id += 1
            primo_prprty +=1
            route_id = map_route_to_division(routes_df, division_id)

            end_date = datetime.now()
            start_date = end_date - timedelta(days=365 * 10)
            first_production_ts = start_date + (end_date - start_date) * random.random()

            facilities.append({
                'facility_id': facility_id,
                'facility_name': f'{county_name} FAC',
                'first_production_ts': first_production_ts,    
                'division_id': division_id,
                'primo_prprty': primo_prprty,
                'route_id': route_id
            })

        start_index = (start_index + batch_size) % county_row_count


    facilities_df = pl.DataFrame(facilities)
    return facilities_df

In [ ]:
from src.pool import PG

timeseries_query = """--sql 
    SELECT *
    FROM await PG.insert(tank_timeseries_with_value2(), "timeseries_dba.timeseries_data").timeseries_data
"""

pp = await PG.fetch(timeseries_query)
if pp is None:
    raise ValueError("df is None")
if pp.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

pp

In [ ]:
def poop():
    
    # Generate the time series data with Perlin noise
    points_per_day = 24
    days = 2
    hours = points_per_day * days

    # Use the fetched DataFrame for time series generation
    initial_df = pp
    noise_dfs = map(
        lambda x: create_ts_df_for_min(initial_df, x, hours), range(hours)
    )

    return noise_dfs

In [ ]:
poop()

In [ ]:
import polars as pl
from datetime import timedelta
from perlin_noise import PerlinNoise

# Initialize Perlin noise generator
random_noise = PerlinNoise(octaves=1)

def create_ts_df_for_min(initial_df: pl.DataFrame, hour: int, total_minutes: int) -> pl.DataFrame:
    offset = timedelta(hours=hour)
    # Generate noise based on timestamp and hour
    noise = random_noise.noise(hour / total_minutes)
    
    # Apply noise to the DataFrame
    noisy_df = initial_df.with_columns(
        [
            pl.col("ts") - offset,
            (pl.col("value") + noise * 100).alias("value")  # Adjust noise magnitude as needed
        ]
    )
    
    return noisy_df

def poop():
    # Define parameters
    points_per_day = 24
    days = 2
    hours = points_per_day * days
    
    # Use the fetched DataFrame for time series generation
    initial_df = pp  # Ensure pp is a Polars DataFrame
    
    if not isinstance(initial_df, pl.DataFrame):
        raise TypeError("pp should be a Polars DataFrame.")
    
    # Generate noise for each hour
    noise_dfs = [create_ts_df_for_min(initial_df, x, hours) for x in range(hours)]
    
    # Combine all generated data into one DataFrame
    combined_df = pl.concat(noise_dfs)
    
    return combined_df

In [ ]:
combined_df

In [ ]:
#new tank timeseries generation

from src.pool import PG 

DATA_CATALOG_QUERY = """--sql
    SELECT *
    FROM timeseries_dba.data_catalog AS dc 
    JOIN timeseries_dba.timeseries_data AS td ON td.key_metric = dc.key_metric
    WHERE dc.metric_nice_name NOT LIKE '%TankID%'
"""

dc = await PG.fetch(DATA_CATALOG_QUERY)
if dc is None:
    raise ValueError("df is None")
if dc.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

dc

In [ ]:
poop()

In [ ]:
from src.pool import PG

DATA_CATALOG_QUERY = """--sql 
    SELECT *
    FROM timeseries_dba.data_catalog dc
    WHERE source_id NOT IN ('69419', '98750', '98743', '480001', '110009')
"""

dc_df = await PG.fetch(DATA_CATALOG_QUERY)
if dc_df is None:
    raise ValueError("df is None")
if dc_df.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

dc_df

In [ ]:
import polars as pl
from src.pool import PG

NEW_TANKS_QUERY = """--sql 
    SELECT DISTINCT source_key, division_id, source_id
    FROM timeseries_dba.data_catalog dc
    WHERE source_id NOT IN ('69419', '98750', '98743', '480001', '110009')
    AND source_key NOT LIKE '%FAC%'
    ORDER BY source_key;
"""

new_tanks_df = await PG.fetch(NEW_TANKS_QUERY)
if new_tanks_df is None:
    raise ValueError("df is None")
if new_tanks_df.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

new_tanks_df
pattern = r'0\d'
oil_tanks_df = new_tanks_df.filter(pl.col("source_key").str.contains(pattern)) 
new_oil_df = oil_tanks_df.group_by("source_id").agg(pl.count("source_key").alias("oil_tank_count"))
new_oil_df = new_oil_df.sort("source_id")
new_oil_df
water_tanks_df = new_tanks_df.filter(pl.col("source_key").str.contains("W"))
new_df = water_tanks_df.group_by("source_id").agg(pl.count("source_key").alias("water_tank_count"))
new_df = new_df.sort("source_id")
new_df
combined_count_df = new_df.join(new_oil_df, on="source_id")
combined_count_df
import polars as pl
pl.Config.set_tbl_rows(2000)
def select_rand_oil(new_df: pl.DataFrame, key_metric):
    oil_count = new_df.with_columns("oil_tank_count").item()
    return oil_count
new_new_df = dc_df.join(combined_count_df, on="source_id")
new_new_df

In [ ]:
import polars as pl
import random
from datetime import datetime, timedelta

def tank_timeseries_with_value3() -> pl.DataFrame:
    time_series = []

    for tank in new_new_df.iter_rows(named=True):
        key_metric = tank["key_metric"]
        primo_id = tank["source_id"]
        identifier = tank["metric_nice_name"]

        if "Inches" in identifier:
            value = random.uniform(60, 400)
        else:
            continue

        # Define the start_date as one week ago and end_date as the current date
        end_date = datetime.now()
        start_date = end_date - timedelta(days=7)  # Set the start date to 1 week ago

        # Generate a random timestamp between start_date and end_date
        create_ts = start_date + (end_date - start_date) * random.random()

        time_series.append({
            "key_metric": key_metric,
            "ts": create_ts,
            "value": value
        })

    time_series_df = pl.DataFrame(time_series)
    return time_series_df


In [ ]:
tank_timeseries_with_value3()